# Gain Statistical Insights into Your DataTable

Woodwork provides two methods on DataTable to allow users to better understand their data: `describe` and `get_mutual_information`

In [ ]:
import pandas as pd
from woodwork import DataTable

df = pd.DataFrame({
        'id': range(3),
        'country': ['AUS', 'GB', 'NZ'],
        'email': ['john.smith@example.com', None, 'team@featuretools.com'],
        'delta': (pd.Series([pd.to_datetime('2020-09-01')] * 3) - pd.to_datetime('2020-07-01')),
        'age': [33, 25, 33],
        'signup_date': [pd.to_datetime('2020-09-01')] * 3,
        'is_registered': [True, False, True],
    })

dt = DataTable(df, index='id')
dt = dt.set_logical_types({'email':'EmailAddress', 'country': 'CountryCode'})

In order to understand which Logical Types `describe` and `get_mutual_information` will include in their calculations, we can split the available Logical Types into five categories:

- Categorical
    - `Categorical`, `CountryCode`, `Ordinal`, `SubRegionCode`, `ZIPCode`
- Numeric
    - `Double`, `Integer`, `WholeNumber`
- String
    - `EmailAddress`, `FilePath`, `FullName`, `IPAddress`, `LatLong`, `NaturalLanguage`, `PhoneNumber`, `URL`
- Boolean - just the `Boolean` LogicalType
- Datetime - just the `Datetime` LogicalType
- Timedelta - just the `Timedelta` LogicalType


## DataTable.describe

Using `dt.describe()` will calculate statistics for the columns in your DataTable and return a Pandas DataFrame with the relevant calculations done for each column.

The statistics calculated can be broken down into a few types:

- General - can be applied to all columns
    - `nan_count` and `mode`
- Aggregate
    - `count` - Categorical, Numeric, Datetime
    - `nunique` - Categorical, Numeric, Datetime
    - `mean` - Numeric, Datetime
    - `std` - Numeric
    - `min` - Numeric, Datetime
    - `max` - Numeric, Datetime
- Boolean - only relevant for columns of Booleans
    - `num_false` and `num_true`
- Quartile - calculated on Numeric columns
    - `first_quartile`
    - `second_quartile`
    - `third_quartile`

In [ ]:
dt.describe()

There are a couple things to note in the above dataframe:
- The DataTable's `index` will not be included
- We provide typing information for each column according to Woodwork's typing system
- Any statistic that cannot be calculated for a column will be filled with `NaN`. 

## DataTable.get_mutual_information()

`dt.get_mutual_information` will calculate the mutual information between all pairs of columns whose Logical Types fall into one of the Numeric, Categorical, or Boolean categories described above.

describe possible inputs
describe the behavior of what happens in terns of -> turning into categories for all?